In [1]:
import requests as rq
import re
from bs4 import BeautifulSoup as bs
import numpy as np
import math
from random import randint
import json
import pickle
import time
# import tqdm
from tqdm import tqdm
from getpass import getpass
import xmltodict, json
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)
pd.set_option('precision', 2)

pd.options.display.float_format = '{:20,.2f}'.format
np.set_printoptions(suppress=False)

# Задание

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [11]:
# Указываем интересующую должность, Headers и Paparams для запросов 
job = 'продавец'

In [12]:
headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36', 'accept': '*/*',
          'cookie': '_xsrf=de884125fde6477ed9c7161ed263ea53; _xsrf=de884125fde6477ed9c7161ed263ea53; hhrole=anonymous; region_clarified=NOT_SET; display=desktop; hhtoken=LPeQjTKZ_afPzaTypkHXCMgn0_jl; hhuid=lk36Wan1dMD2dl74cc1I5w--; GMT=3; _fbp=fb.1.1593340366222.421034606; iap.uid=2016ae8cd7d542b186d6e67a282f7d87; _ga=GA1.2.45243456.1593340367; _gid=GA1.2.1045293019.1593340367; __zzatgib-w-hh=MDA0dBA=Fz2+aQ==; regions=113; __ddg1=arLcdN8Pq8fTWee0ZdlV; _ym_d=1593345354; _ym_uid=159334535496613988; _ym_isad=2; __ddg2=sSwrGRG1bIhS7u2w; total_searches=47; gssc58=; _ym_visorc_156828=w; _gat_gtag_UA_11659974_2=1; cfidsgib-w-hh=yPYHkzwbbl0DO5VJDLh0f+Pjzkb47tVWctu3OAbo9LdPC+lj/9CGbxMAVInqh3ZK/WY/sYxuS1mnY++uDQ+YdSJjEIblX8A66VzewCULumH4nHpBVFhE6T8vOkE3ggxI+Z2Sy33+/mwRkPR7GBSIbk7x2AwQY0EZTjAisLY=; gsscgib-w-hh=NVIvBOyd9CDXzwpLBEvVN1HFf9QmBdIhGDZtZ5ZZ+ps4FZelgYFFK2WM0JAUKnxg2+p9DDGwrfxOPapKqLLkG00k7LEyCdZ8oas9CV4/8dnd8TmJAPbN9OeLhmTQg8vTiauJHU+e8b9PDX2SwgTqQFNmcBN2sahuEv2FuznPrHrndF/3mkhsqTJ4g94=; fgsscgib-w-hh=e0a6861354de74151cc564cc99569903e8742063; cfidsgib-w-hh=cpeq9lhFLTJephPmzT4JIGnNmwmBw8YzbF6rE7KzoX4e/EexNKNVn4RIe5ukHyCGwftXp9d6K1oHMsAoCMnIYJElCYMfks+/vkeg1M26QbfvpepfDBaDFFYKoS47ysDkcgtrpuZAzJYRbqKa8mlaMWF1l97+JMvTQ4jk3Ns='}
params = {'area':113, 'st':'searchVacancy', 'text':job,'fromSearch':True, 'from':'suggest_post', "items_on_page": 100}

*Теперь собираем максильно возможное количество страниц по каждому региону и добавляем ссылки на найденные страницы в список URLS*

In [5]:
areas = [i for i in range(0,5000)]

urls = []
for area in tqdm(areas):
    ur = f"https://hh.ru/search/vacancy?L_is_autosearch=false&area={area}&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text={params['text']}"
    req = rq.get(f"{ur}", headers=headers)
    soup = bs(req.text, 'html.parser')
    try:
        counts = int(str(soup.find_all("h1", attrs={'class':"bloko-header-1"})[0].text.strip()).split('вакансий')[0].replace('\xa0', ''))
    except:
        continue
    if counts > 0 and counts <= 100:
        urls.append([f"https://hh.ru/search/vacancy?L_is_autosearch=true&area={area}&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text={params['text']}&page=0"])
    if counts > 100:
        c = counts // 100 + 1
        res = c if c <=20 else 20
        urls.append([f"https://hh.ru/search/vacancy?L_is_autosearch=true&area={area}&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text={params['text']}&page={i}" for i in list(range(res))])                            

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [48:29<00:00,  2.55it/s]


In [6]:
with open('list_urls_hh_v3.pkl', 'wb') as f:
    pickle.dump(urls, f)

*Теперь собираем по списку URLS все id вакансий и создаем список ссылок LINKS уже на конкретные вакансии*

In [33]:
urls = pd.read_pickle('list_urls_hh_v3.pkl')

In [34]:
links = []
for url in tqdm(urls[:100]):  # собрем не все, а то все - долго ждать
    for href in url:
        print(f"{href}")        
        req = rq.get(f"{href}", headers=headers)
        print(f"Статус: {req.status_code}")     
        soup = bs(req.text, 'html.parser')
        items1 = soup.find_all("script", attrs={"data-name":"HH/Vacancy/SendResponseAttempt"})
        for link in items1:
            data = dict(json.loads(link.get('data-params')))
            links.append(f"https://hh.ru/vacancy/{data['vacancyId']}?query={params['text']}")
links = list(set(links))




  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

https://hh.ru/search/vacancy?L_is_autosearch=true&area=0&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=0
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=0&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=1
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=0&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=2
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=0&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=3
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=0&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=4
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=0&currency_code=RUR&only_with_




  1%|▊                                                                                 | 1/100 [00:24<39:48, 24.13s/it]

https://hh.ru/search/vacancy?L_is_autosearch=true&area=1&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=0
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=1&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=1
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=1&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=2
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=1&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=3
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=1&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=4
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=1&currency_code=RUR&only_with_




  2%|█▋                                                                                | 2/100 [00:44<37:42, 23.09s/it]

...




 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [12:15<00:46, 23.21s/it]

https://hh.ru/search/vacancy?L_is_autosearch=true&area=275&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=0
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=275&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=1
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=275&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=2
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=275&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=3
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=275&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=4
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=275&currency_code=RU




 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [12:39<00:23, 23.43s/it]

https://hh.ru/search/vacancy?L_is_autosearch=true&area=276&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=0
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=276&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=1
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=276&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=2
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=276&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=3
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=276&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text=продавец&page=4
Статус: 200
https://hh.ru/search/vacancy?L_is_autosearch=true&area=276&currency_code=RU




100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [13:04<00:00, 23.97s/it]

In [35]:
with open('final_links.pkl', 'wb') as f:
    pickle.dump(links, f)

***Собираем конечную информацию с отдельных конечных страничек вакансий*** 

In [36]:
links = pd.read_pickle('final_links.pkl')

In [247]:
data_f = []
count_ok = 0
for href in tqdm(links[:100]):
#     time.sleep(randint(2,10))
    print(f"{href}")        
    req = rq.get(f"{href}", headers=headers)
    print(f"Статус: {req.status_code}")     
    soup = bs(req.text, 'html.parser')

    el = {}
    
    v1 = soup.find("script", attrs={'data-name':"HH/GoogleDfpService"})
    if v1:
        dd = v1.get('data-params')    
        salary_min = dict(json.loads(dd))['vac_salary_from']
        salary_max = dict(json.loads(dd))['vac_salary_to']
        lat = np.nan
        lon = np.nan
        employer = soup.find("span", attrs={'class':"bloko-section-header-2 bloko-section-header-2_lite"}).text
        city = soup.find("p", attrs={'data-qa':"vacancy-view-location"}).text
        
        ids1 = soup.find("script", attrs={'data-name':'HH/VacancyResponsePopup/VacancyResponsePopup'})
        if ids1:
            ids = ids1.get('data-params')
            id_point =  dict(json.loads(ids))['vacancyId']
        
        ids2 = soup.find("input", attrs={'name':'vacancyId'})
        if ids2:
            id_point = ids2.get('value')         
        
        currency = dict(json.loads(dd))['vac_salary_cur']
        date1 = soup.find("script", attrs={'type':"application/ld+json"})
        if date1:
            ff = date1.text
            date = dict(json.loads(ff))['datePosted']
            name = dict(json.loads(ff))['title']

        
        date2 = soup.find("span", attrs={'class':"vacancy-serp-item__publication-date"})
        if date2:
            date = date2.text.replace('\xa0', ' ')   
        
        date3 = soup.find("p", attrs={'class':"vacancy-creation-time"})
        if date3:
            date = date3.text.split('Вакансия опубликована')[1].split(' в ')[0].replace('\xa0', ' ').strip()  
        
        
        name = soup.find("h1", attrs={'data-qa':"vacancy-title"}).text
    
    
    v2 = soup.find("script", attrs={'data-name':"HH/Employer/VacancyCard/VacancyMap"})
    if v2:
        dd = v2.get('data-params')    
        salary_min = dict(json.loads(dd))['cianPlacemarks']['compensation']['from']
        salary_max = dict(json.loads(dd))['cianPlacemarks']['compensation']['to']
        lat = dict(json.loads(dd))['address']['mapData']['points']['marker']['lat']
        lon = dict(json.loads(dd))['address']['mapData']['points']['marker']['lng']
        city = dict(json.loads(dd))['address']['city']
        employer = soup.find("span", attrs={'class':"bloko-section-header-2 bloko-section-header-2_lite"}).text
        ids = soup.find("script", attrs={'data-name':"HH/Vacancy/SendResponseAttempt"}).get('data-params')
        id_point = dict(json.loads(ids))['vacancyId']
        currency = dict(json.loads(dd))['cianPlacemarks']['compensation']['currencyCode']
        
        if date:
            pass
        else:
            date = soup.find("span", attrs={'class':"vacancy-serp-item__publication-date"}).text.replace('\xa0', ' ')   
        
        name = soup.find("h1", attrs={'data-qa':"vacancy-title"}).text
    
    el.update({
        'name' : name, 
        'link': href,
        'publish_date': date, 
        'salary_min': salary_min,
        'salary_max': salary_max,
        'lat': lat,
        'lon': lon,
        'city': city,
        'employer': employer,
        'id': id_point,
        'currency' : currency,
        'site': 'hh.ru'

    })
    data_f.append(el)
    count_ok += 1
df = pd.DataFrame(data_f)
hh = df.drop_duplicates('link')
hh.to_pickle('raw_data_v1.pkl')
print('Количество записей ', len(hh), 'Количество обработанных адресов', count_ok)














  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

https://hh.ru/vacancy/37293110?query=продавец
Статус: 200















  1%|▊                                                                                 | 1/100 [00:00<00:38,  2.57it/s]

https://hh.ru/vacancy/37355797?query=продавец
Статус: 200















  2%|█▋                                                                                | 2/100 [00:00<00:37,  2.61it/s]

...

https://hh.ru/vacancy/37452713?query=продавец
Статус: 200















 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:38<00:00,  2.77it/s]

https://hh.ru/vacancy/22089648?query=продавец
Статус: 200















100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:39<00:00,  2.78it/s]

Количество записей  100 Количество обработанных адресов 100


***Готовый DF***

In [ ]:
hh = pd.read_pickle('raw_data_v1.pkl')

In [252]:
hh.sample(25)

,name,link,publish_date,salary_min,salary_max,lat,lon,city,employer,id,currency,site
93,"Продавец-кассир (Жирновск, улица Ломоносова 73)",https://hh.ru/vacancy/37409149?query=продавец,9 июня 2020,25000,0,50.97,44.79,Жирновск,"ООО Бристоль, сеть магазинов",37409149,RUR,hh.ru
31,"Заместитель директора магазина (ул.Паустовского, м. Ясенево)",https://hh.ru/vacancy/37675291?query=продавец,25 июня 2020,47800,54900,55.60,37.54,Москва,Familia,37675291,RUR,hh.ru
9,"Продавец ""Магнит-косметик"" (Мошково)",https://hh.ru/vacancy/37142665?query=продавец,27 июня 2020,17700,,NaN,NaN,Мошково,"МАГНИТ, Розничная сеть",37142665,RUR,hh.ru
98,Продавец-консультант (бытовой техники МБТ и КБТ),https://hh.ru/vacancy/37452713?query=продавец,24 июня 2020,60000,,NaN,NaN,"деревня Чёрная Грязь, Промышленная улица, с2",HOLODILNIK.RU,37452713,RUR,hh.ru
77,Продавец-кассир,https://hh.ru/vacancy/37500970?query=продавец,22 июня 2020,35000,,NaN,NaN,"Советская Гавань, улица Чкалова, 25А",Азбука мебели,37500970,RUR,hh.ru
45,"Старший продавец магазина ""Магнит-Косметик""",https://hh.ru/vacancy/37435033?query=продавец,18 июня 2020,35000,43300,NaN,NaN,Химки,"МАГНИТ, Розничная сеть",37435033,RUR,hh.ru
16,"Продавец-кассир (посёлок городского типа Тереньга, Ульяновская улица, 22)",https://hh.ru/vacancy/37164412?query=продавец,27 июня 2020,18000,24000,53.715663,48.368098,рабочий поселок Тереньга,Пятёрочка. Магазины,37164412,RUR,hh.ru
46,Продавец-кассир,https://hh.ru/vacancy/37687111?query=продавец,26 июня 2020,27500,,NaN,NaN,Хабаровск,ООО Сеть минимаркетов «ЁЁ»,37687111,RUR,hh.ru
96,Продавец-грузчик (м. Багратионовская/Филевский парк/Пионерская),https://hh.ru/vacancy/37608514?query=продавец,26 июня 2020,34500,43100,NaN,NaN,Москва,"МАГНИТ, Розничная сеть",37608514,RUR,hh.ru
90,Продавец-консультант (мкр-н Солнечный ТЦ Байкальский ),https://hh.ru/vacancy/37143772?query=продавец,22 июня 2020,22000,30000,NaN,NaN,"Иркутск, улица Ржанова, 164",ООО Славия Тех,37143772,RUR,hh.ru
